In [1]:
import math
import re
import os
#from queue import Queue, LifoQueue
from collections import deque
from operator import mul
from functools import reduce

import tqdm#.notebook as tqdm

In [2]:
class Factory:
    def __init__(self, blueprint):
        d = [int(x) for x in re.findall("\d+", blueprint)]
        self.id = d[0]
        self.costs = (
            (d[1], 0, 0, 0),
            (d[2], 0, 0, 0),
            (d[3], d[4], 0, 0),
            (d[5], 0, d[6], 0),
        )
        max_ore = max(c[0] for c in self.costs)
        self.max = (max_ore, d[4], d[6], math.inf)
    
    def find_best(self, maxstep=24, tqdm_update=12345):
        q = deque()
        initial = (       # resources, robots, time left
            (0, 0, 0, 0),
            (1, 0, 0, 0),
            maxstep
        )
        q.append(initial)
        c = 0
        best = 0
        checked = {initial}
        with tqdm.tqdm() as progbar:
            while q:
                resources, robots, time_left = q.pop()
                new_res = tuple(resources[k] + robots[k] for k in range(4))
                t = time_left - 1
                if time_left == 0:
                    best = max(best, resources[-1])
                elif new_res[-1] + robots[-1] * time_left + time_left * (time_left-1) // 2 < best:
                    # building one geode robot per turn would not be enough
                    continue
                else:
                    for robot in range(4):
                        if robots[robot] <= self.max[robot] and resources[0] >= self.costs[robot][0]:
                            #all(resources[k] >= self.costs[robot][k] for k in range(3)):
                            if (robot == 2 and resources[1] < self.costs[robot][1]) or (robot == 3 and resources[2] < self.costs[robot][2]):
                                continue
                            new_res_b = tuple(new_res[k] - self.costs[robot][k] for k in range(3)) + (new_res[-1],)
                            new_robots = robots[:robot] + (robots[robot] + 1,) + robots[robot+1:]
                            new_state = (new_res_b, new_robots, t)
                            if new_state not in checked:
                                q.append(new_state)
                                checked.add(new_state)
                    new_state = (new_res, robots, t) # do not build
                    if new_state not in checked:
                        q.append(new_state)
                        checked.add(new_state)

                c += 1
                if c % tqdm_update == 0:
                    progbar.update(tqdm_update)
                    progbar.set_description(f"Blueprint #{self.id} Path #{c}, timesteps left: {time_left:02d}, queue: {len(q):02d}")

        return best


def part1(factories):
    print(len(factories), "blueprints found")
    tot = 0
    for f in factories:
        quality = f.id * f.find_best(24)
        #print(f"#{f.id}: {quality}")
        tot += quality
    return tot

def quality(f):
    return f.id * f.find_best(20)

# def part1_parallel(factories):
#     pool = Pool(2)
#     quality_levels = pool.map(quality, factories)
#     return sum(quality_levels)

def part2(factories):
    best_runs = [f.find_best(32) for f in factories[:3]]
    return reduce(mul, best_runs, 1)

In [3]:
with open("19_input.txt") as f:
    factories = [Factory(line.rstrip()) for line in f]

In [4]:
part1(factories)

30 blueprints found


Blueprint #1 Path #1333260, timesteps left: 03, queue: 06: : 1333260it [00:04, 302728.78it/s]
Blueprint #2 Path #592560, timesteps left: 02, queue: 05: : 592560it [00:01, 389956.25it/s]
Blueprint #3 Path #641940, timesteps left: 07, queue: 07: : 641940it [00:02, 270720.82it/s]
Blueprint #4 Path #1654230, timesteps left: 05, queue: 06: : 1654230it [00:06, 268821.83it/s]
Blueprint #5 Path #2641830, timesteps left: 04, queue: 06: : 2641830it [00:10, 257332.57it/s]
Blueprint #6 Path #1740645, timesteps left: 05, queue: 11: : 1740645it [00:06, 287765.24it/s]
Blueprint #7 Path #1074015, timesteps left: 02, queue: 10: : 1074015it [00:03, 323382.20it/s]
Blueprint #8 Path #3061560, timesteps left: 02, queue: 08: : 3061560it [00:09, 333150.38it/s]
Blueprint #9 Path #1345605, timesteps left: 07, queue: 10: : 1345605it [00:05, 259197.07it/s]
Blueprint #10 Path #777735, timesteps left: 04, queue: 05: : 777735it [00:03, 254631.33it/s]
Blueprint #11 Path #2036925, timesteps left: 02, queue: 11: : 203

1404

In [5]:
part2(factories)

Blueprint #1 Path #20653185, timesteps left: 09, queue: 10: : 20653185it [02:50, 121127.56it/s]
Blueprint #2 Path #14246130, timesteps left: 05, queue: 09: : 14246130it [01:19, 179620.13it/s]
Blueprint #3 Path #9690825, timesteps left: 11, queue: 05: : 9690825it [01:03, 152402.85it/s]


5880